In [1]:

# Load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import read_csv, set_option
from pandas.plotting import scatter_matrix
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

#Libraries for Deep Learning Models
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD

In [6]:
# load dataset
dataset = pd.read_csv('bitstampUSD_1-min_data_2012-01-01_to_2020-12-31.csv')

In [7]:
dataset.shape

(4727777, 8)

In [8]:
# peek at data
set_option('display.width', 100)
dataset.tail(2)

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
4727775,1609372740,28910.54,28911.52,28867.60,28881.30,7.332773,211870.91266,28893.695831
4727776,1609372800,28893.21,28928.49,28893.21,28928.49,5.757679,166449.70932,28909.166061


In [9]:
#NANを補完
dataset[dataset.columns.values] = dataset[dataset.columns.values].ffill()

In [11]:
# Create short simple moving average over the short window
dataset['short_mavg'] = dataset['Close'].rolling(window=10, min_periods=1,\
center=False).mean()
# Create long simple moving average over the long window
dataset['long_mavg'] = dataset['Close'].rolling(window=60, min_periods=1,\
center=False).mean()
# Create signals
dataset['signal'] = np.where(dataset['short_mavg'] >
dataset['long_mavg'], 1.0, 0.0)

In [13]:
#calculation of exponential moving average
def EMA(df, n):
    EMA = pd.Series(df['Close'].ewm(span=n, min_periods=n).mean(), name='EMA_'+ str(n))
    return EMA

dataset['EMA10'] = EMA(dataset, 10)
dataset['EMA30'] = EMA(dataset, 30)
dataset['EMA200'] = EMA(dataset, 200)
dataset.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,short_mavg,long_mavg,signal,EMA10,EMA30,EMA200
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39,4.39,4.39,0.0,NaN,NaN,NaN
1,1325317980,4.39,4.39,4.39,4.39,0.455581,2.0,4.39,4.39,4.39,0.0,NaN,NaN,NaN
2,1325318040,4.39,4.39,4.39,4.39,0.455581,2.0,4.39,4.39,4.39,0.0,NaN,NaN,NaN
3,1325318100,4.39,4.39,4.39,4.39,0.455581,2.0,4.39,4.39,4.39,0.0,NaN,NaN,NaN
4,1325318160,4.39,4.39,4.39,4.39,0.455581,2.0,4.39,4.39,4.39,0.0,NaN,NaN,NaN
